## Problema #3

### Formulación del Modelo

* Índices

$$
i = 1,2,3,4\quad (\text{trabajadores}), 
\quad j = 1,2,3,4\quad (\text{puestos}).
$$

* Parámetros

$$
c_{ij} = 
\begin{pmatrix}
50 & 50 & - & 20\\
70 & 40 & 20 & 30\\
90 & 30 & 50 & -\\
70 & 20 & 60 & 70
\end{pmatrix},
$$

donde $c_{1,3}=+\infty$ (trabajador 1 no puede ir al puesto 3) y
$c_{3,4}=+\infty$ (trabajador 3 no puede ir al puesto 4).

* Variables de decisión

$$
x_{ij} =
\begin{cases}
1 & \text{si al trabajador }i\text{ se le asigna el puesto }j,\\
0 & \text{en otro caso.}
\end{cases}
$$

* Función objetivo (minimizar coste total)

$$
\min \sum_{i=1}^4\sum_{j=1}^4 c_{ij}\,x_{ij}.
$$

* Restricciones

1. Cada trabajador hace exactamente un puesto:
   $\displaystyle\sum_{j=1}^4 x_{ij} = 1\quad \forall\,i=1,\dots,4.$
2. Cada puesto es cubierto por exactamente un trabajador:
   $\displaystyle\sum_{i=1}^4 x_{ij} = 1\quad \forall\,j=1,\dots,4.$
3. $x_{ij}\in\{0,1\}$.

---

### Solución

In [1]:
import JuMP
import HiGHS
import Ipopt
import Optimization

using JuMP
using HiGHS
using Ipopt
using Optimization

In [10]:
# Costos
costs_p3 = [
    50 50 1e6 20;
    70 40 20 30;
    90 30 50 1e6;
    70 20 60 70
]

4×4 Matrix{Float64}:
 50.0  50.0   1.0e6  20.0
 70.0  40.0  20.0    30.0
 90.0  30.0  50.0     1.0e6
 70.0  20.0  60.0    70.0

In [12]:
# Modelo
model = Model(HiGHS.Optimizer)

# Variables 
@variable(model, x[1:4, 1:4], Bin)

# Objetivo
@objective(model, Min, sum(costs_p3[i, j] * x[i, j] for i in 1:4, j in 1:4))

# Restricciones

# Cada trabajador i asignado a un único puesto
@constraint(model,  [i in 1:4], sum(x[i, j] for j in 1:4) == 1)

# Cada puesto j ocupado por un único trabajador
@constraint(model,  [j in 1:4], sum(x[i, j] for i in 1:4) == 1)

4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1,1] + x[2,1] + x[3,1] + x[4,1] == 1
 x[1,2] + x[2,2] + x[3,2] + x[4,2] == 1
 x[1,3] + x[2,3] + x[3,3] + x[4,3] == 1
 x[1,4] + x[2,4] + x[3,4] + x[4,4] == 1

In [13]:
# Resolver
optimize!(model)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 8 rows; 16 cols; 32 nonzeros; 16 integer variables (16 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+01, 1e+06]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
8 rows, 16 cols, 32 nonzeros  0s
8 rows, 16 cols, 32 nonzeros  0s
Objective function is integral with scale 0.1

Solving MIP model with:
   8 rows
   16 cols (16 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   32 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraint

In [14]:
# Resultados
println("Coste óptimo: \$", objective_value(model))
for i in 1:4, j in 1:4
    if value(x[i, j]) > 0.5
        println("Trabajador $i -> Puesto $j (coste=", costs_p3[i, j], ")")
    end
end

Coste óptimo: $140.0
Trabajador 1 -> Puesto 4 (coste=20.0)
Trabajador 2 -> Puesto 3 (coste=20.0)
Trabajador 3 -> Puesto 2 (coste=30.0)
Trabajador 4 -> Puesto 1 (coste=70.0)
